In [19]:
import pickle as pkl
import numpy as np
import torch
from utils import *

In [17]:
dataset = 'multi30k'
embed_key = 'embed_beta'
dat = load_dataset_dict(dataset, embed_key)
metric = TopPercentMatch(p=5)
# metric = KendallTauCorr()
runs = ['normal_B']

In [21]:
print(fetch_stats(dat, 'val', metric, 'normal_A', runs, 'uniform', embed_key, 'val_acc'))

{'agr_unif': 0.31019777503090235, 'agr_px': 0.3442521631644005, 'agr_grad': 0.27058096415327565, 'xi_unif': 43.4487021013597, 'xi_px': 43.4487021013597, 'xi_grad': 43.4487021013597, 'best_perf': 66.29171817058096, 'baseline': 0.07137206427688506}


In [2]:
dataset = 'multi30k'
dat = pkl.load(open('outputs/{dataset}_logs.pkl'.format(dataset=dataset), 'rb'))
all_dicts = dat['data']
subset = 'val'
acc_metric = 'val_bleu'
dicts = [d for d in all_dicts if d['split'] == subset]
iterations = sorted(list(set([key[1] for key in dat['metas']])))
# flip_grads(dicts)

In [3]:
embed_beta = pkl.load(open('outputs/{dataset}embedding256translation.pkl'.format(dataset=dataset), 'rb'))
impute_beta(dicts, embed_beta, 'embed_beta')

In [4]:
metric = TopPercentMatch(p=5)
normalA_iter = max_acc_iter('normal_A', dat['metas'], acc_metric)
normalB_iter = max_acc_iter('normal_B', dat['metas'], acc_metric)
uniform_iter = max_acc_iter('uniform', dat['metas'], acc_metric)

In [5]:
gold_alpha_key = ('normal_A', normalA_iter, 'alpha')
gold_grad_key = ('normal_A', normalA_iter, 'grad')

normal_keys = [('normal_B', iter_, 'alpha') for iter_ in iterations]
acc_keys = [('normal_B', iter_, acc_metric) for iter_ in iterations]

alpha_corrs, alpha_baseline = corrs_iter(dicts, gold_alpha_key, normal_keys, metric)
alpha_perfs = acc_iter(dat['metas'], acc_keys)

avg_corr = np.array(alpha_corrs)
avg_perf = np.array(alpha_perfs)
baseline = alpha_baseline[0]

In [6]:
print(dicts[0][gold_grad_key].shape)
print(dicts[0][gold_alpha_key].shape)

(14, 14)
(14, 14)


In [7]:
beta_unif_keys = [('uniform', iter_, 'beta') for iter_ in iterations]
beta_corr_unif = max_corr(dicts, gold_alpha_key, beta_unif_keys, metric)
beta_corr_grad = max_corr(dicts, gold_grad_key, beta_unif_keys, metric)
beta_corr_px = max_corr(dicts, 'embed_beta', beta_unif_keys, metric)

In [8]:
print(corrs_iter(dicts, gold_grad_key, beta_unif_keys, metric))

([0.06508034610630407, 0.05760197775030902, 0.06464771322620519, 0.14573547589616812, 0.18590852904820768, 0.20970333745364647, 0.22632880098887515, 0.25173053152039554, 0.25630407911001235, 0.2656365883807169, 0.26458590852904823, 0.2671817058096415, 0.26613102595797283, 0.2686650185414092, 0.26526576019777504, 0.26817058096415325, 0.26637824474660077, 0.26909765142150804, 0.2723733003708282, 0.2622373300370828], [0.06909765142150803, 0.06835599505562423, 0.06971569839307787, 0.07132262051915945, 0.07169344870210136, 0.0700247218788628, 0.07218788627935722, 0.07224969097651422, 0.07280593325092707, 0.0738566131025958, 0.06878862793572312, 0.07682323856613102, 0.07286773794808406, 0.07799752781211372, 0.07546353522867738, 0.07398022249690976, 0.07855377008652657, 0.08139678615574783, 0.08176761433868975, 0.08479604449938195])


In [15]:
# for iter_ in iterations:
#     alpha_key = ('normal_A', iter_, 'alpha')
#     grad_key = ('normal_A', iter_, 'grad')
#     print(iter_, metric.eval_corr(dicts, alpha_key, grad_key))

0 {'name': 'top 5% match', 'correlation': 0.07428924598269468, 'baseline': 0.07169344870210136}
50 {'name': 'top 5% match', 'correlation': 0.23454882571075403, 'baseline': 0.07336217552533993}
100 {'name': 'top 5% match', 'correlation': 0.39641532756489495, 'baseline': 0.07163164400494437}
500 {'name': 'top 5% match', 'correlation': 0.37626699629171817, 'baseline': 0.07169344870210136}
1000 {'name': 'top 5% match', 'correlation': 0.3983930778739184, 'baseline': 0.07262051915945612}
1500 {'name': 'top 5% match', 'correlation': 0.4251545117428925, 'baseline': 0.06953028430160692}
2000 {'name': 'top 5% match', 'correlation': 0.4566749072929543, 'baseline': 0.07169344870210136}
4000 {'name': 'top 5% match', 'correlation': 0.46983930778739186, 'baseline': 0.06903584672435105}
6000 {'name': 'top 5% match', 'correlation': 0.4711372064276885, 'baseline': 0.07262051915945612}
8000 {'name': 'top 5% match', 'correlation': 0.46946847960444993, 'baseline': 0.07200247218788627}
10000 {'name': 'top 5

In [10]:
best_acc = dat['metas'][('normal_A', normalA_iter, acc_metric)]
idx_unif = passing_idx(avg_corr, beta_corr_unif)
idx_grad = passing_idx(avg_corr, beta_corr_grad)
idx_px = passing_idx(avg_corr, beta_corr_px)
def print_perf(idx):
    if idx is None:
        return None
    else:
        return avg_perf[idx]
print(list(zip(iterations, avg_corr)))
print(idx_unif, idx_grad, idx_px)
print(beta_corr_unif, beta_corr_px, beta_corr_grad, baseline, print_perf(idx_unif), print_perf(idx_px), print_perf(idx_grad), best_acc)



[(0, 0.06328800988875155), (50, 0.0634734239802225), (100, 0.16452410383189123), (500, 0.26674907292954264), (1000, 0.45327564894932015), (1500, 0.5652657601977751), (2000, 0.630655129789864), (4000, 0.7021631644004944), (6000, 0.715327564894932), (8000, 0.7527194066749073), (10000, 0.7559950556242274), (12000, 0.7480222496909765), (14000, 0.75), (16000, 0.7639678615574784), (18000, 0.7688504326328801), (20000, 0.7632262051915946), (24000, 0.776761433868974), (28000, 0.7726823238566131), (32000, 0.7765760197775031), (36000, 0.7749690976514215)]
4 4 4
0.307725587144623 0.3442521631644005 0.2723733003708282 0.0715080346106304 11.428225554342985 11.428225554342985 11.428225554342985 37.88620336527713


In [31]:
# metrics.append(SpearmanRankCorr()) comment out because this takes a lot of time to evaluate
key1, key2 = ('normal_A', normalA_iter, 'alpha'), ('normal_B', normalB_iter, 'beta')
print(key1, key2)
for metric in metrics:
    print(metric.eval_corr(dicts, key1, key2))

('normal_A', 98000, 'alpha') ('normal_B', 98000, 'beta')
{'name': 'top 1 match', 'correlation': 0.4906809735378769, 'baseline': 0.029517236240347983}
{'name': 'top 3 match', 'correlation': 0.630424455807975, 'baseline': 0.08793696034872998}
{'name': 'top 5 match', 'correlation': 0.6835303322435521, 'baseline': 0.1448337850802433}
{'name': 'top 5% match', 'correlation': 0.6016337403712707, 'baseline': 0.06422257616825476}


In [32]:
key1, key2 = ('normal_A', iterations[-1], 'alpha'), ('uniform', iterations[-1], 'beta')
print(key1, key2)
for metric in metrics:
    print(metric.eval_corr(dicts, key1, key2))

('normal_A', 98000, 'alpha') ('uniform', 98000, 'beta')
{'name': 'top 1 match', 'correlation': 0.26383882415402243, 'baseline': 0.031557083112117065}
{'name': 'top 3 match', 'correlation': 0.3738182393979192, 'baseline': 0.0881372192881959}
{'name': 'top 5 match', 'correlation': 0.43500433118171405, 'baseline': 0.14561619209955198}
{'name': 'top 5% match', 'correlation': 0.34707202801762277, 'baseline': 0.06557315971349}


In [33]:
key1, key2 = ('uniform', iterations[-1], 'beta'), 'embed256_beta'
print(key1, key2)
for metric in metrics:
    print(metric.eval_corr(dicts, key1, key2))

('uniform', 98000, 'beta') embed256_beta
{'name': 'top 1 match', 'correlation': 0.34697888432484797, 'baseline': 0.040349847710062316}
{'name': 'top 3 match', 'correlation': 0.41936084798017903, 'baseline': 0.09972895185402521}
{'name': 'top 5 match', 'correlation': 0.47952701632808936, 'baseline': 0.1557176255809838}
{'name': 'top 5% match', 'correlation': 0.39109173722301394, 'baseline': 0.07727200752601038}


In [34]:
key1, key2 = ('uniform', iterations[-1], 'beta'), 'IBM_beta'
print(key1, key2)
for metric in metrics:
    print(metric.eval_corr(dicts, key1, key2))

('uniform', 98000, 'beta') IBM_beta


KeyError: 'IBM_beta'